In [1]:
from collections import defaultdict

# Grammar rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,
    ('VP', 'V', 'NP'): 0.7,
    ('VP', 'VP', 'PP'): 0.3,
    ('PP', 'P', 'NP'): 1.0,
    ('NP', 'astronomers'): 0.1,
    ('NP', 'ears'): 0.18,
    ('NP', 'stars'): 0.18,
    ('NP', 'telescopes'): 0.18,
    ('V', 'saw'): 1.0,
    ('P', 'with'): 1.0
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform CYK algorithm
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)
    # Create a table to store probabilities
    table = defaultdict(float)
    
    # Initialize for the single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:
                table[(i, i, rule[0])] = pcfg[rule]
    
    # Filling the table for larger spans (length > 1)
    for span in range(2, n+1):  # span length
        for i in range(n - span + 1):  # starting point of the span
            j = i + span - 1  # ending point of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rules
                        A, B, C = rule
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            prob = table[(i, k, B)] * table[(k + 1, j, C)] * pcfg[rule]
                            if prob > table[(i, j, A)]:
                                table[(i, j, A)] = prob

    # Print the table with inside probabilities
    return table

# Run the CYK algorithm
table = cyk_algorithm(pcfg, sentence)

# Print the resulting probabilities
for key, prob in table.items():
    print(f"Span {key[0]}-{key[1]} -> {key[2]}: Probability = {prob:.6f}")



Span 0-0 -> NP: Probability = 0.100000
Span 1-1 -> V: Probability = 1.000000
Span 2-2 -> NP: Probability = 0.180000
Span 3-3 -> P: Probability = 1.000000
Span 4-4 -> NP: Probability = 0.180000
Span 1-2 -> VP: Probability = 0.126000
Span 3-4 -> PP: Probability = 0.180000
Span 0-2 -> S: Probability = 0.012600
Span 1-4 -> VP: Probability = 0.006804
Span 0-4 -> S: Probability = 0.000680


In [2]:
import numpy as np
from collections import defaultdict

class PCFG:
    def __init__(self):
        # Non-terminal production rules and their probabilities
        self.productions = defaultdict(list)
        self.terminals = defaultdict(list)

    def add_production(self, lhs, rhs, prob):
        """ Adds a production rule with its probability """
        if len(rhs) == 1 and rhs[0].islower():  # Terminal rule
            self.terminals[rhs[0]].append((lhs, prob))
        else:  # Non-terminal rule
            self.productions[lhs].append((rhs, prob))

def cyk_pcfg(pcfg, words):
    """ Applies the CYK algorithm to find the inside probability of a word sequence """
    n = len(words)
    non_terminals = list(pcfg.productions.keys())
    
    # Initialize a 3D table for inside probabilities
    P = defaultdict(lambda: np.zeros((n, n)))
    
    # Fill the diagonal with terminal production probabilities
    for i, word in enumerate(words):
        if word in pcfg.terminals:
            for lhs, prob in pcfg.terminals[word]:
                P[lhs][i, i] = prob

    # Fill the table for subsequences
    for span in range(2, n + 1):  # span length from 2 to n
        for i in range(n - span + 1):
            j = i + span - 1
            for k in range(i, j):  # midpoint
                for lhs in non_terminals:
                    for rhs, prob in pcfg.productions[lhs]:
                        if len(rhs) == 2:
                            left, right = rhs
                            P[lhs][i, j] += prob * P[left][i, k] * P[right][k + 1, j]

    # The inside probability for the start symbol S to derive the entire sequence
    return P['S'][0, n - 1]

# Example Usage:

# Define a PCFG
pcfg = PCFG()
pcfg.add_production('S', ['NP', 'VP'], 0.9)
pcfg.add_production('S', ['VP'], 0.1)
pcfg.add_production('NP', ['Det', 'N'], 0.5)
pcfg.add_production('VP', ['V', 'NP'], 0.5)
pcfg.add_production('VP', ['eats'], 0.1)
pcfg.add_production('Det', ['the'], 0.8)
pcfg.add_production('N', ['cat'], 0.5)
pcfg.add_production('N', ['food'], 0.5)
pcfg.add_production('V', ['eats'], 1.0)

# Example word sequence
words = ['the', 'cat', 'eats']

# Calculate inside probability using CYK algorithm
inside_prob = cyk_pcfg(pcfg, words)

print(f"Inside probability of the sequence {' '.join(words)}: {inside_prob}")



Inside probability of the sequence the cat eats: 0.018000000000000002


In [3]:
from collections import defaultdict

# Grammar rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,
    ('VP', 'V', 'NP'): 0.7,
    ('VP', 'VP', 'PP'): 0.3,
    ('PP', 'P', 'NP'): 1.0,
    ('NP', 'astronomers'): 0.1,
    ('NP', 'ears'): 0.18,
    ('NP', 'stars'): 0.18,
    ('NP', 'telescopes'): 0.18,
    ('V', 'saw'): 1.0,
    ('P', 'with'): 1.0
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform CYK algorithm
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)
    # Create a table to store probabilities
    table = defaultdict(float)
    
    # Initialize for the single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:
                table[(i, i, rule[0])] = pcfg[rule]
    
    # Filling the table for larger spans (length > 1)
    for span in range(2, n+1):  # span length
        for i in range(n - span + 1):  # starting point of the span
            j = i + span - 1  # ending point of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rules
                        A, B, C = rule
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            prob = table[(i, k, B)] * table[(k + 1, j, C)] * pcfg[rule]
                            if prob > table[(i, j, A)]:
                                table[(i, j, A)] = prob

    # Return the final result for the whole sentence
    return table[(0, n-1, 'S')]  # The probability of the sentence being an S (sentence)

# Run the CYK algorithm
final_prob = cyk_algorithm(pcfg, sentence)

# Print the final probability of the sentence
print(f"Final Probability of the sentence: {final_prob:.6f}")


Final Probability of the sentence: 0.000680


In [9]:
from collections import defaultdict

# Probabilistic context-free grammar (PCFG) rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,         # S -> NP VP
    ('VP', 'V', 'NP'): 0.7,         # VP -> V NP
    ('VP', 'VP', 'PP'): 0.3,        # VP -> VP PP
    ('PP', 'P', 'NP'): 1.0,         # PP -> P NP
    ('NP', 'astronomers'): 0.1,     # NP -> astronomers
    ('NP', 'ears'): 0.18,           # NP -> ears
    ('NP', 'stars'): 0.18,          # NP -> stars
    ('NP', 'telescopes'): 0.18,     # NP -> telescopes
    ('V', 'saw'): 1.0,              # V -> saw
    ('P', 'with'): 1.0              # P -> with
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform the CYK algorithm and calculate inside probabilities
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)  # Length of the sentence (number of words)
    
    # Table to store probabilities: (start_index, end_index, non-terminal) -> probability
    table = defaultdict(float)
    
    # Step 1: Initialize the table for single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:  # Match terminal rules like NP -> astronomers
                table[(i, i, rule[0])] = pcfg[rule]
    
    # Step 2: Fill the table for larger spans (length > 1)
    for span in range(2, n + 1):  # span length
        for i in range(n - span + 1):  # start index of the span
            j = i + span - 1  # end index of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rule like S -> NP VP
                        A, B, C = rule  # A -> B C
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            prob = table[(i, k, B)] * table[(k + 1, j, C)] * pcfg[rule]
                            if prob > table[(i, j, A)]:
                                table[(i, j, A)] = prob

    # Step 3: Return the final result for the whole sentence as an 'S' (complete sentence)
    # The final probability for the entire sentence to be an S should be in table[(0, n-1, 'S')]
    return table[(0, n-1, 'S')]  # Probability of the whole sentence being an S (sentence)

# Run the CYK algorithm and get the final probability
final_prob = cyk_algorithm(pcfg, sentence)

# Print the final probability of the sentence
if final_prob > 0:
    print(f"Final Probability of the sentence: {final_prob:.6f}")
else:
    print("The sentence could not be parsed with the given grammar.")

Final Probability of the sentence: 0.000680


In [11]:
from collections import defaultdict

# Probabilistic context-free grammar (PCFG) rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,         # S -> NP VP
    ('VP', 'V', 'NP'): 0.7,         # VP -> V NP
    ('VP', 'VP', 'PP'): 0.3,        # VP -> VP PP
    ('PP', 'P', 'NP'): 1.0,         # PP -> P NP
    ('NP', 'astronomers'): 0.1,     # NP -> astronomers
    ('NP', 'ears'): 0.18,           # NP -> ears
    ('NP', 'stars'): 0.18,          # NP -> stars
    ('NP', 'telescopes'): 0.18,     # NP -> telescopes
    ('V', 'saw'): 1.0,              # V -> saw
    ('P', 'with'): 1.0              # P -> with
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform the CYK algorithm and calculate inside probabilities
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)  # Length of the sentence (number of words)
    
    # Table to store probabilities: (start_index, end_index, non-terminal) -> list of (prob, derivation)
    table = defaultdict(list)
    
    # Step 1: Initialize the table for single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:  # Match terminal rules like NP -> astronomers
                table[(i, i, rule[0])].append((pcfg[rule], word))
    
    # Step 2: Fill the table for larger spans (length > 1)
    for span in range(2, n + 1):  # span length
        for i in range(n - span + 1):  # start index of the span
            j = i + span - 1  # end index of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rule like S -> NP VP
                        A, B, C = rule  # A -> B C
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            for prob1, derivation1 in table[(i, k, B)]:
                                for prob2, derivation2 in table[(k + 1, j, C)]:
                                    prob = prob1 * prob2 * pcfg[rule]
                                    table[(i, j, A)].append((prob, (B, derivation1, C, derivation2)))

    # Step 3: Return the list of possible derivations for the whole sentence as 'S' (complete sentence)
    # The final probability for the entire sentence to be an S should be in table[(0, n-1, 'S')]
    return table[(0, n-1, 'S')]  # List of all parses (each with a probability and derivation)

# Run the CYK algorithm and get all possible parses
parses = cyk_algorithm(pcfg, sentence)

# Print the final probabilities and derivations of the sentence
if parses:
    for idx, (prob, derivation) in enumerate(parses, start=1):
        print(f"Parse t{idx}: Probability = {prob:.6f}, Derivation = {derivation}")
else:
    print("The sentence could not be parsed with the given grammar.")


Parse t1: Probability = 0.000680, Derivation = ('NP', 'astronomers', 'VP', ('VP', ('V', 'saw', 'NP', 'stars'), 'PP', ('P', 'with', 'NP', 'ears')))


In [14]:
from collections import defaultdict

# Probabilistic context-free grammar (PCFG) rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,         # S -> NP VP
    ('VP', 'V', 'NP'): 0.7,         # VP -> V NP
    ('VP', 'VP', 'PP'): 0.3,        # VP -> VP PP
    ('PP', 'P', 'NP'): 1.0,         # PP -> P NP
    ('NP', 'astronomers'): 0.1,     # NP -> astronomers
    ('NP', 'ears'): 0.18,           # NP -> ears
    ('NP', 'stars'): 0.18,          # NP -> stars
    ('NP', 'telescopes'): 0.18,     # NP -> telescopes
    ('V', 'saw'): 1.0,              # V -> saw
    ('P', 'with'): 1.0              # P -> with
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform the CYK algorithm and calculate inside probabilities
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)  # Length of the sentence (number of words)
    
    # Table to store probabilities: (start_index, end_index, non-terminal) -> list of (prob, derivation)
    table = defaultdict(list)
    
    # Step 1: Initialize the table for single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:  # Match terminal rules like NP -> astronomers
                table[(i, i, rule[0])].append((pcfg[rule], word))
    
    # Step 2: Fill the table for larger spans (length > 1)
    for span in range(2, n + 1):  # span length
        for i in range(n - span + 1):  # start index of the span
            j = i + span - 1  # end index of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rule like S -> NP VP
                        A, B, C = rule  # A -> B C
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            for prob1, derivation1 in table[(i, k, B)]:
                                for prob2, derivation2 in table[(k + 1, j, C)]:
                                    prob = prob1 * prob2 * pcfg[rule]
                                    table[(i, j, A)].append((prob, (B, derivation1, C, derivation2)))

    # Step 3: Return the list of possible derivations for the whole sentence as 'S' (complete sentence)
    # The final probability for the entire sentence to be an S should be in table[(0, n-1, 'S')]
    return table[(0, n-1, 'S')]  # List of all parses (each with a probability and derivation)

# Run the CYK algorithm and get all possible parses
parses = cyk_algorithm(pcfg, sentence)

# Print the final probabilities and derivations of the sentence
if parses:
    for idx, (prob, derivation) in enumerate(parses, start=1):
        print(f"Parse t{idx}: Probability = {prob:.6f}, Derivation = {derivation}")
else:
    print("The sentence could not be parsed with the given grammar.")


Parse t1: Probability = 0.000680, Derivation = ('NP', 'astronomers', 'VP', ('VP', ('V', 'saw', 'NP', 'stars'), 'PP', ('P', 'with', 'NP', 'ears')))


In [19]:
from collections import defaultdict

# Probabilistic context-free grammar (PCFG) rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,         # S -> NP VP
    ('VP', 'V', 'NP'): 0.7,         # VP -> V NP
    ('VP', 'VP', 'PP'): 0.3,        # VP -> VP PP
    ('PP', 'P', 'NP'): 1.0,         # PP -> P NP
    ('NP', 'astronomers'): 0.1,     # NP -> astronomers
    ('NP', 'ears'): 0.18,           # NP -> ears
    ('NP', 'stars'): 0.18,          # NP -> stars
    ('NP', 'telescopes'): 0.18,     # NP -> telescopes
    ('V', 'saw'): 1.0,              # V -> saw
    ('P', 'with'): 1.0              # P -> with
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform the CYK algorithm and calculate inside probabilities
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)  # Length of the sentence (number of words)
    
    # Table to store probabilities: (start_index, end_index, non-terminal) -> list of (prob, derivation)
    table = defaultdict(list)
    
    # Step 1: Initialize the table for single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:  # Match terminal rules like NP -> astronomers
                table[(i, i, rule[0])].append((pcfg[rule], word))
    
    # Step 2: Fill the table for larger spans (length > 1)
    for span in range(2, n + 1):  # span length
        for i in range(n - span + 1):  # start index of the span
            j = i + span - 1  # end index of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rule like S -> NP VP
                        A, B, C = rule  # A -> B C
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            for prob1, derivation1 in table[(i, k, B)]:
                                for prob2, derivation2 in table[(k + 1, j, C)]:
                                    prob = prob1 * prob2 * pcfg[rule]
                                    table[(i, j, A)].append((prob, (B, derivation1, C, derivation2)))

    # Step 3: Return the list of possible derivations for the whole sentence as 'S' (complete sentence)
    # The final probability for the entire sentence to be an S should be in table[(0, n-1, 'S')]
    return table[(0, n-1, 'S')]  # List of all parses (each with a probability and derivation)

# Run the CYK algorithm and get all possible parses
parses = cyk_algorithm(pcfg, sentence)

# Print the final probabilities and derivations of the sentence
if parses:
    for idx, (prob, derivation) in enumerate(parses, start=1):
        print(f"Parse t{idx}: Probability = {prob:.6f}, Derivation = {derivation}")
    #print(parses)
else:
    print("The sentence could not be parsed with the given grammar.")


Parse t1: Probability = 0.000680, Derivation = ('NP', 'astronomers', 'VP', ('VP', ('V', 'saw', 'NP', 'stars'), 'PP', ('P', 'with', 'NP', 'ears')))


In [1]:
from collections import defaultdict

# Probabilistic context-free grammar (PCFG) rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,         # S -> NP VP
    ('VP', 'V', 'NP'): 0.7,         # VP -> V NP
    ('VP', 'VP', 'PP'): 0.3,        # VP -> VP PP
    ('PP', 'P', 'NP'): 1.0,         # PP -> P NP
    ('NP', 'astronomers'): 0.1,     # NP -> astronomers
    ('NP', 'ears'): 0.18,           # NP -> ears
    ('NP', 'stars'): 0.18,          # NP -> stars
    ('NP', 'telescopes'): 0.18,     # NP -> telescopes
    ('V', 'saw'): 1.0,              # V -> saw
    ('P', 'with'): 1.0              # P -> with
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform the CYK algorithm and calculate inside probabilities
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)  # Length of the sentence (number of words)
    
    # Table to store probabilities: (start_index, end_index, non-terminal) -> list of (prob, derivation)
    table = defaultdict(list)
    
    # Step 1: Initialize the table for single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:  # Match terminal rules like NP -> astronomers
                table[(i, i, rule[0])].append((pcfg[rule], word))
    
    # Step 2: Fill the table for larger spans (length > 1)
    for span in range(2, n + 1):  # span length
        for i in range(n - span + 1):  # start index of the span
            j = i + span - 1  # end index of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rule like S -> NP VP
                        A, B, C = rule  # A -> B C
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            for prob1, derivation1 in table[(i, k, B)]:
                                for prob2, derivation2 in table[(k + 1, j, C)]:
                                    prob = prob1 * prob2 * pcfg[rule]
                                    table[(i, j, A)].append((prob, (B, derivation1, C, derivation2)))

    # Step 3: Return the list of possible derivations for the whole sentence as 'S' (complete sentence)
    # The final probability for the entire sentence to be an S should be in table[(0, n-1, 'S')]
    return table[(0, n-1, 'S')]  # List of all parses (each with a probability and derivation)

# Helper function to print the parse tree in a readable format
def print_parse_tree(derivation, indent=0):
    if isinstance(derivation, tuple):
        A, derivation1, B, derivation2 = derivation
        print(' ' * indent + f"({A}")
        print_parse_tree(derivation1, indent + 2)
        print_parse_tree(derivation2, indent + 2)
        print(' ' * indent + f")")
    else:
        print(' ' * indent + derivation)

# Run the CYK algorithm and get all possible parses
parses = cyk_algorithm(pcfg, sentence)

# Print the final probabilities and derivations of the sentence
if parses:
    for idx, (prob, derivation) in enumerate(parses, start=1):
        print(f"Parse t{idx}: Probability = {prob:.6f}")
        print_parse_tree(derivation)
        print()  # Print a blank line between parses
else:
    print("The sentence could not be parsed with the given grammar.")


Parse t1: Probability = 0.000680
(NP
  astronomers
  (VP
    (V
      saw
      stars
    )
    (P
      with
      ears
    )
  )
)



In [2]:
from collections import defaultdict

# Probabilistic context-free grammar (PCFG) rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,         # S -> NP VP
    ('VP', 'V', 'NP'): 0.7,         # VP -> V NP
    ('VP', 'VP', 'PP'): 0.3,        # VP -> VP PP
    ('PP', 'P', 'NP'): 1.0,         # PP -> P NP
    ('NP', 'astronomers'): 0.1,     # NP -> astronomers
    ('NP', 'ears'): 0.18,           # NP -> ears
    ('NP', 'stars'): 0.18,          # NP -> stars
    ('NP', 'telescopes'): 0.18,     # NP -> telescopes
    ('V', 'saw'): 1.0,              # V -> saw
    ('P', 'with'): 1.0              # P -> with
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform the CYK algorithm and calculate inside probabilities
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)  # Length of the sentence (number of words)
    
    # Table to store probabilities: (start_index, end_index, non-terminal) -> list of (prob, derivation)
    table = defaultdict(list)
    
    # Step 1: Initialize the table for single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:  # Match terminal rules like NP -> astronomers
                table[(i, i, rule[0])].append((pcfg[rule], word))
    
    # Step 2: Fill the table for larger spans (length > 1)
    for span in range(2, n + 1):  # span length
        for i in range(n - span + 1):  # start index of the span
            j = i + span - 1  # end index of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rule like S -> NP VP
                        A, B, C = rule  # A -> B C
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            for prob1, derivation1 in table[(i, k, B)]:
                                for prob2, derivation2 in table[(k + 1, j, C)]:
                                    prob = prob1 * prob2 * pcfg[rule]
                                    table[(i, j, A)].append((prob, (B, derivation1, C, derivation2)))

    # Step 3: Return the list of possible derivations for the whole sentence as 'S' (complete sentence)
    # The final probability for the entire sentence to be an S should be in table[(0, n-1, 'S')]
    return table[(0, n-1, 'S')]  # List of all parses (each with a probability and derivation)

# Helper function to print the parse tree in a readable format
def print_parse_tree(derivation, indent=0):
    if isinstance(derivation, tuple):
        A, derivation1, B, derivation2 = derivation
        print(' ' * indent + f"({A}")
        print_parse_tree(derivation1, indent + 2)
        print_parse_tree(derivation2, indent + 2)
        print(' ' * indent + f")")
    else:
        print(' ' * indent + derivation)

# Run the CYK algorithm and get all possible parses
parses = cyk_algorithm(pcfg, sentence)

# Sort parses by probability in descending order
parses.sort(key=lambda x: x[0], reverse=True)

# Print the final probabilities and derivations of the sentence
if parses:
    for idx, (prob, derivation) in enumerate(parses, start=1):
        print(f"Parse t{idx}: Probability = {prob:.6f}")
        print_parse_tree(derivation)
        print()  # Print a blank line between parses
else:
    print("The sentence could not be parsed with the given grammar.")


Parse t1: Probability = 0.000680
(NP
  astronomers
  (VP
    (V
      saw
      stars
    )
    (P
      with
      ears
    )
  )
)



In [3]:
from collections import defaultdict

# Probabilistic context-free grammar (PCFG) rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,         # S -> NP VP
    ('VP', 'V', 'NP'): 0.7,         # VP -> V NP
    ('VP', 'VP', 'PP'): 0.3,        # VP -> VP PP
    ('PP', 'P', 'NP'): 1.0,         # PP -> P NP
    ('NP', 'astronomers'): 0.1,     # NP -> astronomers
    ('NP', 'ears'): 0.18,           # NP -> ears
    ('NP', 'stars'): 0.18,          # NP -> stars
    ('NP', 'telescopes'): 0.18,     # NP -> telescopes
    ('V', 'saw'): 1.0,              # V -> saw
    ('P', 'with'): 1.0              # P -> with
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform the CYK algorithm and calculate inside probabilities
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)  # Length of the sentence (number of words)
    
    # Table to store probabilities: (start_index, end_index, non-terminal) -> list of (prob, derivation)
    table = defaultdict(list)
    
    # Step 1: Initialize the table for single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:  # Match terminal rules like NP -> astronomers
                table[(i, i, rule[0])].append((pcfg[rule], rule[1]))
    
    # Step 2: Fill the table for larger spans (length > 1)
    for span in range(2, n + 1):  # span length
        for i in range(n - span + 1):  # start index of the span
            j = i + span - 1  # end index of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rule like S -> NP VP
                        A, B, C = rule  # A -> B C
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            for prob1, derivation1 in table[(i, k, B)]:
                                for prob2, derivation2 in table[(k + 1, j, C)]:
                                    prob = prob1 * prob2 * pcfg[rule]
                                    table[(i, j, A)].append((prob, (B, derivation1, C, derivation2)))

    # Step 3: Return the list of possible derivations for the whole sentence as 'S' (complete sentence)
    # The final probability for the entire sentence to be an S should be in table[(0, n-1, 'S')]
    return table[(0, n-1, 'S')]  # List of all parses (each with a probability and derivation)

# Helper function to convert the parse tree into a string
def build_parse_tree(derivation):
    if isinstance(derivation, tuple):
        A, derivation1, B, derivation2 = derivation
        return f"({A} {build_parse_tree(derivation1)} {build_parse_tree(derivation2)})"
    else:
        return derivation

# Run the CYK algorithm and get all possible parses
parses = cyk_algorithm(pcfg, sentence)

# Function to sort parses by probability (optional, for better readability)
def sort_parses(parses):
    return sorted(parses, key=lambda x: x[0], reverse=True)

# Sort the parses (optional)
sorted_parses = sort_parses(parses)

# Print the final probabilities and derivations of the sentence
if sorted_parses:
    for idx, (prob, derivation) in enumerate(sorted_parses, start=1):
        tree_str = build_parse_tree(derivation)
        print(f"Parse t{idx}: Probability = {prob:.6f}")
        print(tree_str)
        print()  # Print a blank line between parses
else:
    print("The sentence could not be parsed with the given grammar.")


Parse t1: Probability = 0.000680
(NP astronomers (VP (V saw stars) (P with ears)))



In [4]:
from collections import defaultdict

# Probabilistic context-free grammar (PCFG) rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,         # S -> NP VP
    ('VP', 'V', 'NP'): 0.7,         # VP -> V NP
    ('VP', 'VP', 'PP'): 0.3,        # VP -> VP PP
    ('PP', 'P', 'NP'): 1.0,         # PP -> P NP
    ('NP', 'astronomers'): 0.1,     # NP -> astronomers
    ('NP', 'ears'): 0.18,           # NP -> ears
    ('NP', 'stars'): 0.18,          # NP -> stars
    ('NP', 'telescopes'): 0.18,     # NP -> telescopes
    ('V', 'saw'): 1.0,              # V -> saw
    ('P', 'with'): 1.0              # P -> with
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform the CYK algorithm and calculate inside probabilities
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)  # Length of the sentence (number of words)
    
    # Table to store probabilities: (start_index, end_index, non-terminal) -> list of (prob, derivation)
    table = defaultdict(list)
    
    # Step 1: Initialize the table for single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:  # Match terminal rules like NP -> astronomers
                table[(i, i, rule[0])].append((pcfg[rule], word))
    
    # Step 2: Fill the table for larger spans (length > 1)
    for span in range(2, n + 1):  # span length
        for i in range(n - span + 1):  # start index of the span
            j = i + span - 1  # end index of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rule like S -> NP VP
                        A, B, C = rule  # A -> B C
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            for prob1, derivation1 in table[(i, k, B)]:
                                for prob2, derivation2 in table[(k + 1, j, C)]:
                                    prob = prob1 * prob2 * pcfg[rule]
                                    table[(i, j, A)].append((prob, (B, derivation1, C, derivation2)))

    # Step 3: Return the list of possible derivations for the whole sentence as 'S' (complete sentence)
    # The final probability for the entire sentence to be an S should be in table[(0, n-1, 'S')]
    return table[(0, n-1, 'S')]  # List of all parses (each with a probability and derivation)

# Helper function to print the parse tree in a readable format
def print_parse_tree(derivation, indent=0):
    if isinstance(derivation, tuple):
        A, derivation1, B, derivation2 = derivation
        print(' ' * indent + f"({A}")
        print_parse_tree(derivation1, indent + 2)
        print_parse_tree(derivation2, indent + 2)
        print(' ' * indent + f")")
    else:
        print(' ' * indent + derivation)

# Run the CYK algorithm and get all possible parses
parses = cyk_algorithm(pcfg, sentence)

# Sort parses by probability in descending order
parses.sort(key=lambda x: x[0], reverse=True)

# Print the final probabilities and derivations of the sentence
if parses:
    for idx, (prob, derivation) in enumerate(parses, start=1):
        print(f"Parse t{idx}: Probability = {prob:.6f}")
        print_parse_tree(derivation)
        print()  # Print a blank line between parses
else:
    print("The sentence could not be parsed with the given grammar.")

Parse t1: Probability = 0.000680
(NP
  astronomers
  (VP
    (V
      saw
      stars
    )
    (P
      with
      ears
    )
  )
)



In [2]:
from collections import defaultdict

# Probabilistic context-free grammar (PCFG) rules with probabilities
pcfg = {
    ('S', 'NP', 'VP'): 1.0,         # S -> NP VP
    ('VP', 'V', 'NP'): 0.7,         # VP -> V NP
    ('VP', 'VP', 'PP'): 0.3,        # VP -> VP PP
    ('PP', 'P', 'NP'): 1.0,         # PP -> P NP
    ('NP', 'astronomers'): 0.1,     # NP -> astronomers
    ('NP', 'ears'): 0.18,           # NP -> ears
    ('NP', 'stars'): 0.18,          # NP -> stars
    ('NP', 'telescopes'): 0.18,     # NP -> telescopes
    ('V', 'saw'): 1.0,              # V -> saw
    ('P', 'with'): 1.0              # P -> with
}

# The sentence we want to parse
sentence = "astronomers saw stars with ears".split()

# Function to perform the CYK algorithm and calculate inside probabilities
def cyk_algorithm(pcfg, sentence):
    n = len(sentence)  # Length of the sentence (number of words)
    
    # Table to store probabilities
    table = defaultdict(float)
    backpointer = defaultdict(lambda: None)

    # Step 1: Initialize the table for single words (length 1 spans)
    for i, word in enumerate(sentence):
        for rule in pcfg:
            if len(rule) == 2 and rule[1] == word:  # Match terminal rules like NP -> astronomers
                table[(i, i, rule[0])] = pcfg[rule]

    # Step 2: Fill the table for larger spans (length > 1)
    for span in range(2, n + 1):  # span length
        for i in range(n - span + 1):  # start index of the span
            j = i + span - 1  # end index of the span
            for k in range(i, j):  # split point
                for rule in pcfg:
                    if len(rule) == 3:  # binary rule like S -> NP VP
                        A, B, C = rule  # A -> B C
                        if (i, k, B) in table and (k + 1, j, C) in table:
                            prob = table[(i, k, B)] * table[(k + 1, j, C)] * pcfg[rule]
                            if prob > table[(i, j, A)]:
                                table[(i, j, A)] = prob
                                backpointer[(i, j, A)] = (B, C, i, k, j)

    # Step 3: Return the final result for the whole sentence as an 'S'
    return table[(0, n-1, 'S')], backpointer

# Function to build the parse tree from the backpointer
def build_parse_tree(backpointer, i, j, A):
    if (i, j, A) not in backpointer or backpointer[(i, j, A)] is None:
        return A  # Base case: return the non-terminal if no children

    B, C, left_start, split, right_end = backpointer[(i, j, A)]
    left_tree = build_parse_tree(backpointer, left_start, split, B)
    right_tree = build_parse_tree(backpointer, split + 1, right_end, C)
    return f'({A} {left_tree} {right_tree})'

# Run the CYK algorithm and get the final probability and backpointer
final_prob, backpointer = cyk_algorithm(pcfg, sentence)

# Print the final probability of the sentence
if final_prob > 0:
    print(f"Final Probability of the sentence: {final_prob:.6f}")
    parse_tree = build_parse_tree(backpointer, 0, len(sentence) - 1, 'S')
    print("Parse Tree:", parse_tree)
else:
    print("The sentence could not be parsed with the given grammar.")

Final Probability of the sentence: 0.000680
Parse Tree: (S NP (VP (VP V NP) (PP P NP)))
